In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
df=pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [5]:
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [6]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
label_encoder_gender=LabelEncoder()
df['Gender']=label_encoder_gender.fit_transform(df['Gender'])

In [35]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619.0,0.0,42.0,2.0,0.00,1.0,1.0,1.0,101348.88,1.0,1.0,0.0,0.0
1,608.0,0.0,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58,0.0,0.0,0.0,1.0
2,502.0,0.0,42.0,8.0,159660.80,3.0,1.0,0.0,113931.57,1.0,1.0,0.0,0.0
3,699.0,0.0,39.0,1.0,0.00,2.0,0.0,0.0,93826.63,0.0,1.0,0.0,0.0
4,850.0,0.0,43.0,2.0,125510.82,1.0,1.0,1.0,79084.10,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
9996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
9997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
9998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0


In [11]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(df[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [13]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [14]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [21]:
geo_encoder_df=pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [23]:
df=pd.concat([df,geo_encoder_df],axis=1)

In [24]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Geography_France,Geography_Germany,Geography_Spain
0,619.0,0.0,42.0,2.0,0.00,1.0,1.0,1.0,101348.88,1.0,NaN,NaN,NaN,1.0,0.0,0.0
1,608.0,0.0,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58,0.0,NaN,NaN,NaN,0.0,0.0,1.0
2,502.0,0.0,42.0,8.0,159660.80,3.0,1.0,0.0,113931.57,1.0,NaN,NaN,NaN,1.0,0.0,0.0
3,699.0,0.0,39.0,1.0,0.00,2.0,0.0,0.0,93826.63,0.0,NaN,NaN,NaN,1.0,0.0,0.0
4,850.0,0.0,43.0,2.0,125510.82,1.0,1.0,1.0,79084.10,0.0,NaN,NaN,NaN,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,0.0
9996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,0.0
9997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,0.0
9998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,1.0,0.0


In [27]:
# Drop the old encoded columns from df if they exist
df = df.drop(['Geography_France', 'Geography_Germany', 'Geography_Spain'], axis=1, errors='ignore')
df = pd.concat([df, geo_encoder_df], axis=1)


In [28]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619.0,0.0,42.0,2.0,0.00,1.0,1.0,1.0,101348.88,1.0,1.0,0.0,0.0
1,608.0,0.0,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58,0.0,0.0,0.0,1.0
2,502.0,0.0,42.0,8.0,159660.80,3.0,1.0,0.0,113931.57,1.0,1.0,0.0,0.0
3,699.0,0.0,39.0,1.0,0.00,2.0,0.0,0.0,93826.63,0.0,1.0,0.0,0.0
4,850.0,0.0,43.0,2.0,125510.82,1.0,1.0,1.0,79084.10,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
9996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
9997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
9998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0


In [29]:
print(geo_encoder_df.shape)  # Check the shape of geo_encoder_df


(10000, 3)


In [30]:
print("Before concatenation:", df.shape)
df = pd.concat([df, geo_encoder_df], axis=1)
print("After concatenation:", df.shape)


Before concatenation: (20000, 13)
After concatenation: (20000, 16)


In [31]:
df = df.drop(['Geography_France', 'Geography_Germany', 'Geography_Spain'], axis=1, errors='ignore')
df = pd.concat([df, geo_encoder_df], axis=1)


In [34]:
df.shape

(20000, 13)

In [36]:
df = df.drop_duplicates().reset_index(drop=True)


In [37]:
df.shape

(10003, 13)

In [38]:
df.tail()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
9998,772.0,1.0,42.0,3.0,75075.31,2.0,1.0,0.0,92888.52,1.0,0.0,1.0,0.0
9999,792.0,0.0,28.0,4.0,130142.79,1.0,1.0,0.0,38190.78,0.0,1.0,0.0,0.0
10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
10001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0
10002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0


In [39]:
df = df.iloc[:10000].reset_index(drop=True)
print(df.shape)


(10000, 13)


In [40]:
df.tail()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
9995,771.0,1.0,39.0,5.0,0.00,2.0,1.0,0.0,96270.64,0.0,1.0,0.0,0.0
9996,516.0,1.0,35.0,10.0,57369.61,1.0,1.0,1.0,101699.77,0.0,1.0,0.0,0.0
9997,709.0,0.0,36.0,7.0,0.00,1.0,0.0,1.0,42085.58,1.0,1.0,0.0,0.0
9998,772.0,1.0,42.0,3.0,75075.31,2.0,1.0,0.0,92888.52,1.0,0.0,1.0,0.0
9999,792.0,0.0,28.0,4.0,130142.79,1.0,1.0,0.0,38190.78,0.0,1.0,0.0,0.0


In [41]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619.0,0.0,42.0,2.0,0.00,1.0,1.0,1.0,101348.88,1.0,1.0,0.0,0.0
1,608.0,0.0,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58,0.0,0.0,0.0,1.0
2,502.0,0.0,42.0,8.0,159660.80,3.0,1.0,0.0,113931.57,1.0,1.0,0.0,0.0
3,699.0,0.0,39.0,1.0,0.00,2.0,0.0,0.0,93826.63,0.0,1.0,0.0,0.0
4,850.0,0.0,43.0,2.0,125510.82,1.0,1.0,1.0,79084.10,0.0,0.0,0.0,1.0


In [42]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [44]:
X=df.drop('Exited',axis=1)
y=df['Exited']

X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.2,random_state=42)

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [45]:
X_train

array([[-0.38381126, -1.12244688,  2.60786056, ...,  1.00400803,
        -0.57427105, -0.58350885],
       [-0.73723819, -1.12244688,  0.12136034, ..., -0.99600797,
         1.74133801, -0.58350885],
       [ 0.32304261,  0.89091075, -0.54808203, ...,  1.00400803,
        -0.57427105, -0.58350885],
       ...,
       [ 0.85318301, -1.12244688, -0.06990891, ...,  1.00400803,
        -0.57427105, -0.58350885],
       [ 0.14632915,  0.89091075,  0.40826421, ...,  1.00400803,
        -0.57427105, -0.58350885],
       [ 0.45817644,  0.89091075,  1.1733412 , ..., -0.99600797,
         1.74133801, -0.58350885]])

In [47]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [51]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [52]:
(X_train.shape[1],)

(12,)

In [54]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),  ##HL1 connected with input layer
    Dense(32,activation='relu'), ##HL2
    Dense(1,activation='sigmoid') ##output layer
])

In [55]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [56]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

In [57]:
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=["accuracy"])

In [59]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="logs/fit"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [62]:
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [63]:
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
63/63 [==============================] - 1s 10ms/step - loss: 0.3194 - accuracy: 0.8625 - val_loss: 0.4103 - val_accuracy: 0.8349
Epoch 2/100
63/63 [==============================] - 1s 9ms/step - loss: 0.3257 - accuracy: 0.8625 - val_loss: 0.3726 - val_accuracy: 0.8466
Epoch 3/100
63/63 [==============================] - 1s 10ms/step - loss: 0.3084 - accuracy: 0.8725 - val_loss: 0.3899 - val_accuracy: 0.8479
Epoch 4/100
63/63 [==============================] - 1s 11ms/step - loss: 0.3113 - accuracy: 0.8705 - val_loss: 0.3912 - val_accuracy: 0.8469
Epoch 5/100
63/63 [==============================] - 1s 12ms/step - loss: 0.3067 - accuracy: 0.8705 - val_loss: 0.3872 - val_accuracy: 0.8486
Epoch 6/100
63/63 [==============================] - 1s 13ms/step - loss: 0.2928 - accuracy: 0.8805 - val_loss: 0.4023 - val_accuracy: 0.8484
Epoch 7/100
63/63 [==============================] - 1s 16ms/step - loss: 0.2894 - accuracy: 0.8840 - val_loss: 0.4016 - val_accuracy: 0.8430
Epoch 8

In [64]:
model.save('model.h5')

d:\DATA SCIENCE & ML\DEEP LEARNING\ANN CLASSIFICATION\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [69]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [70]:
%tensorboard --logdir logs/fit20250514-234527

Reusing TensorBoard on port 6007 (pid 2340), started 0:01:07 ago. (Use '!kill 2340' to kill it.)